In [6]:
import os 
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'
import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.training import checkpoints
import matplotlib.pyplot as plt
from tifffile import imread, imsave
import torch
import torch.utils.data as dt

os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP-JAX')
from gap_jax.GAP_UNET_ResBlock_JAX_v2 import UN
os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP/gap')
from GAP_UNET_ResBlock import UN as UN2

In [7]:

tabulate_fn = nn.tabulate(UN(levels=10, channels= 1, depth= 6, start_filts= 28, up_mode= 'upsample', merge_mode= 'concat'), jax.random.PRNGKey(0),
                          console_kwargs={'width': 150})
img = jnp.ones((1, 256, 256, 1))
print(tabulate_fn(img))


                                                            UN Summary                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                           ┃ module     ┃ inputs                  ┃ outputs                 ┃ params                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                                │ UN         │ float32[1,256,256,1]    │ float32[1,256,256,1]    │                              │
├────────────────────────────────┼────────────┼─────────────────────────┼─────────────────────────┼──────────────────────────────┤
│ down_convs_0                   │ DownConv   │ float32[1,256,256,10]   │ - float32[1,128,128,28] │                              │
│                                │            │                         │ - float3

In [8]:
model = UN2(channels = 1, levels=10, depth=6, start_filts=28, up_mode = 'upsample', merge_mode = 'concat')
from torchsummary import summary
summary(model.cuda(), input_size = (1, 256, 256), batch_size=1)

/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP/gap/GAP_UNET_ResBlock.py:258: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP/gap/GAP_UNET_ResBlock.py:259: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 28, 256, 256]           2,548
            Conv2d-2          [1, 28, 256, 256]           7,084
            Conv2d-3          [1, 28, 256, 256]           7,084
         MaxPool2d-4          [1, 28, 128, 128]               0
          DownConv-5  [[-1, 28, 128, 128], [-1, 28, 256, 256]]               0
            Conv2d-6          [1, 56, 128, 128]          14,168
            Conv2d-7          [1, 56, 128, 128]          28,280
            Conv2d-8          [1, 56, 128, 128]          28,280
         MaxPool2d-9            [1, 56, 64, 64]               0
         DownConv-10  [[-1, 56, 64, 64], [-1, 56, 128, 128]]               0
           Conv2d-11           [1, 112, 64, 64]          56,560
           Conv2d-12           [1, 112, 64, 64]         113,008
           Conv2d-13           [1, 112, 64, 64]         113,008
        Max